<a href="https://colab.research.google.com/github/SethupathyM/Supervised-ML---Regression/blob/main/Poly_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing
import numpy as np
import pandas as pd
from numpy import math
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [2]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/Mydrive')

Mounted at /content/Mydrive


In [28]:
# reading the data
data = pd.read_csv('/content/Mydrive/MyDrive/AlmaBetter/Modules/Machine Learning/Projects/Copy of SeoulBikeData.csv',encoding= 'unicode_escape')

In [29]:
data['Temperature'] = data['Temperature(°C)'] + data['Dew point temperature(°C)']
data.drop('Temperature(°C)',axis = 1,inplace = True)
data.drop('Dew point temperature(°C)',axis = 1,inplace = True)

In [30]:
# OneHotEncoding
data = pd.get_dummies(data, columns=['Seasons', 'Holiday', 'Functioning Day','Hour'], prefix=['Season', 'Holiday','Functioning Day','Hour'])

In [32]:
data.drop(['Season_Autumn','Holiday_Holiday','Functioning Day_No','Hour_23'],axis = 1,inplace = True)

In [40]:
final_indep = list(data.columns)
final_indep.remove('Date')
final_indep.remove('Rented Bike Count')

final_dep = ['Rented Bike Count']
print(final_indep)
print(final_dep)
x = data[final_indep].values
y = data[final_dep].values
print(len(final_indep))

['Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Temperature', 'Season_Spring', 'Season_Summer', 'Season_Winter', 'Holiday_No Holiday', 'Functioning Day_Yes', 'Hour_0', 'Hour_1', 'Hour_2', 'Hour_3', 'Hour_4', 'Hour_5', 'Hour_6', 'Hour_7', 'Hour_8', 'Hour_9', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14', 'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_20', 'Hour_21', 'Hour_22']
['Rented Bike Count']
35


In [53]:
# Train_Test_Split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 0)
# Transforming data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [54]:
# Poly Regression
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2)
x_train_poly = poly_features.fit_transform(x_train)
poly_model = LinearRegression()
poly_model.fit(x_train_poly, y_train)
y_train_predicted = poly_model.predict(x_train_poly)
y_test_predict = poly_model.predict(poly_features.fit_transform(x_test))

In [55]:
from sklearn.metrics import r2_score
print('Train R2:',r2_score(y_train,y_train_predicted))
print('Test R2:',r2_score(y_test, y_test_predict))
print("Adjusted R2 : ",1-(1-r2_score(y_test, y_test_predict))*((x_test.shape[0]-1)/(x_test.shape[0]-x_test.shape[1]-1)))

Train R2: 0.8440246346072366
Test R2: 0.8316676130823694
Adjusted R2 :  0.8282342602023478


In [37]:
# Lasso
from sklearn.linear_model import Lasso
lasso  = Lasso(alpha=0.0001 , max_iter= 3000)
lasso.fit(x_train_poly, y_train)
lasso.score(x_train_poly, y_train)
y_train_predicted_l = poly_model.predict(x_train_poly)
y_test_predict_l = poly_model.predict(poly_features.fit_transform(x_test))
print('Train R2:',r2_score(y_train,y_train_predicted_l))
print('Test R2:',r2_score(y_test, y_test_predict_l))
print("Adjusted R2 : ",1-(1-r2_score(y_test, y_test_predict_l))*((x_test.shape[0]-1)/(x_test.shape[0]-x_test.shape[1]-1)))

Train R2: 0.844024717875719
Test R2: 0.8316679388592417
Adjusted R2 :  0.8282345926238533


In [38]:
# Ridge
from sklearn.linear_model import Ridge
ridge  = Ridge(alpha=0.0001 , max_iter= 3000)
ridge.fit(x_train_poly, y_train)
ridge.score(x_train_poly, y_train)
y_train_predicted_r = poly_model.predict(x_train_poly)
y_test_predict_r = poly_model.predict(poly_features.fit_transform(x_test))
print('Train R2:',r2_score(y_train,y_train_predicted_r))
print('Test R2:',r2_score(y_test, y_test_predict_r))
print("Adjusted R2 : ",1-(1-r2_score(y_test, y_test_predict_r))*((x_test.shape[0]-1)/(x_test.shape[0]-x_test.shape[1]-1)))

Train R2: 0.844024717875719
Test R2: 0.8316679388592417
Adjusted R2 :  0.8282345926238533


In [39]:
# Cross Validation for Lasso
from sklearn.model_selection import GridSearchCV
lasso = Lasso()
parameters = {'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100,0.0014]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=5)
lasso_regressor.fit(x_train_poly, y_train)
y_train_predicted_lasso = lasso_regressor.predict(x_train_poly)
y_test_predict_lasso = lasso_regressor.predict(poly_features.fit_transform(x_test))
mean_squared_error(y_test, y_test_predict_lasso)
from sklearn.metrics import r2_score
print(r2_score(y_test, y_test_predict_lasso))

0.7801747289151287


In [24]:
# Cross Validation for Ridge
from sklearn.linear_model import Ridge
ridge = Ridge()
parameters = {'alpha': [1e-15,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1,5,10,20,30,40,45,50,55,60,100]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=3)
ridge_regressor.fit(x_train_poly,y_train)
y_train_predicted_ridge = ridge_regressor.predict(x_train_poly)
y_test_predict_ridge  = ridge_regressor.predict(poly_features.fit_transform(x_test))
mean_squared_error(y_test, y_test_predict_ridge )
from sklearn.metrics import r2_score
print(r2_score(y_test, y_test_predict_ridge ))

0.6936546728036774


In [26]:
# ElasticNetRegression
from sklearn.linear_model import ElasticNet
#a * L1 + b * L2
#alpha = a + b and l1_ratio = a / (a + b)
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.5)
elasticnet.fit(x_train_poly,y_train)
elasticnet.score(x_train_poly, y_train)
y_pred_en = elasticnet.predict(poly_features.fit_transform(x_test))
mean_squared_error(y_test, y_pred_en)
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred_en))

0.6912473231909327


In [45]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
# Number of trees
n_estimators = [25,40,50]

# Maximum depth of trees
max_depth = [2,3,4]

# Minimum number of samples required to split a node
min_samples_split = [25,50,75]

# Minimum number of samples required at each leaf node
min_samples_leaf = [20,25]

# HYperparameter Grid
param_dict = {'n_estimators' : n_estimators,
              'max_depth' : max_depth,
              'min_samples_split' : min_samples_split,
              'min_samples_leaf' : min_samples_leaf}
rf_model = RandomForestClassifier()
rf_grid = GridSearchCV(estimator=rf_model,
                       param_grid = param_dict,
                       cv = 5, verbose=2, scoring='roc_auc')
rf_grid.fit(x_train_poly,y_train)
rf_grid.best_estimator_
rf_optimal_model = rf_grid.best_estimator_
train_class_preds = rf_optimal_model.predict(x_train_poly)
test_class_preds = rf_optimal_model.predict(poly_features.fit_transform(x_test))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=25; total time=   1.3s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=25; total time=   1.7s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=25; total time=   0.9s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=25; total time=   0.9s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=25; total time=   0.9s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=40; total time=   1.6s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=40; total time=   1.4s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=40; total time=   1.6s
[CV] END max_depth=2, min_samples_leaf=20, min_samples_split=25, n_estimators=40; total time=   1.5s
[CV] END max_depth=2, min_sam

In [46]:
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix
train_accuracy = accuracy_score(y_train,train_class_preds)
test_accuracy = accuracy_score(y_test,test_class_preds)
print(train_accuracy)
print(test_accuracy)

0.03724315068493151
0.041666666666666664


In [41]:
import pandas as pd
from sklearn.metrics import accuracy_score, auc
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [51]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=350)
humidity_classifier = DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=50, random_state=350)
humidity_classifier.fit(x_train_poly, y_train)
y_predicted_dec = humidity_classifier.predict(poly_features.fit_transform(x_test))
y_train_predicted_dec = humidity_classifier.predict(x_train_poly)


In [52]:
print(accuracy_score(y_train, y_train_predicted_dec))
print(accuracy_score(y_test, y_predicted_dec))

0.03667237442922375
0.0228310502283105
